In [1]:
#install addons for 
!pip install tensorflow-addons
!pip install transformers[tf-cpu]

from transformers import BertTokenizer, TFBertModel 

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf

import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


from tensorflow.keras.layers import Embedding, Dense, Input, Dropout, LayerNormalization, GlobalAveragePooling1D, Flatten
from tensorflow_addons.layers import MultiHeadAttention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import csv 
import os
import shutil



In [2]:
# Download the data from Kaggle https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/tasks?taskId=588 
# Load the 1mdb datasets from Kaggle
kaggle_imdb_file = 'datasets/IMDB_Dataset.csv' 
data_x = []
data_y = []
with open(kaggle_imdb_file, 'r') as csvfile: 
  filereader = csv.reader(csvfile, delimiter=',', dialect='excel')
  next(filereader)
  for row in filereader:
    data_x.append(row[0]) 
    label = 1 if row[1] == 'positive' else 0 
    data_y.append(label)
# Prepare the data into trainable format
data_x = np.array(data_x)
data_y = np.array(data_y)
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=29)

In [3]:
# Generate Pre-trained outputs 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

n_dim = 768 
def compute_bert_pt(text_data, batch_size): 
  i = 0; 
  output = tf.reshape(tf.constant([]), (0, n_dim))
  while i*batch_size < text_data.shape[0]:
    start = i*batch_size 
    end = (i+1)*batch_size
    if end > text_data.shape[0]: 
      end = text_data.shape[0] 
    batch_pt = bert_model(tokenizer(text_data[start:end].tolist(), max_length = 512, pad_to_max_length=True, truncation=True, return_tensors='tf'))['pooler_output'] 
    output = tf.concat([output, batch_pt], 0)
    print(f'Processing sample {start} to {end} ')
    i+=1
  return output

X_train_pt = compute_bert_pt(X_train, 100)
X_test_pt = compute_bert_pt(X_test, 100) 


     |████████████████████████████████| 2.6 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 738 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 30.2 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 96 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 172.4 MB 66 kB/s s eta 0:00:01    |███████████████▍                | 83.1 MB 39.6 MB/s eta 0:00:03
     |████████████████████████████████| 12.3 MB 15.3 MB/s eta 0:00:01    |███▊                            | 1.4 MB 15.3 MB/s eta 0:00:01     |██████▌                         | 2.5 MB 15.3 MB/s eta 0:00:01     |██████████████                  | 5.3 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 

  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=f1e52b11777b36867b3211db2bc63bb6893ecbd808a2c2258b5cc60c7db4b69c
  Stored in directory: /home/jovyan/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30705 sha256=fecca56ea5c29e6b15241cfb38b0472fc8c8dbdc9075355ef804069bce8402e2
  Stored in directory: /home/jovyan/.cache/pip/wheels/f1/60/77/22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
Successfully built fire clang
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2184: FutureWarning: The `pad_to_max_length` argument is deprecated and will be remo

Processing sample 0 to 100 
Processing sample 100 to 200 
Processing sample 200 to 300 
Processing sample 300 to 400 
Processing sample 400 to 500 
Processing sample 500 to 600 
Processing sample 600 to 700 
Processing sample 700 to 800 
Processing sample 800 to 900 
Processing sample 900 to 1000 
Processing sample 1000 to 1100 
Processing sample 1100 to 1200 
Processing sample 1200 to 1300 
Processing sample 1300 to 1400 
Processing sample 1400 to 1500 
Processing sample 1500 to 1600 
Processing sample 1600 to 1700 
Processing sample 1700 to 1800 
Processing sample 1800 to 1900 
Processing sample 1900 to 2000 
Processing sample 2000 to 2100 
Processing sample 2100 to 2200 
Processing sample 2200 to 2300 
Processing sample 2300 to 2400 
Processing sample 2400 to 2500 
Processing sample 2500 to 2600 
Processing sample 2600 to 2700 
Processing sample 2700 to 2800 
Processing sample 2800 to 2900 
Processing sample 2900 to 3000 
Processing sample 3000 to 3100 
Processing sample 3100 to 320

Processing sample 24800 to 24900 
Processing sample 24900 to 25000 
Processing sample 25000 to 25100 
Processing sample 25100 to 25200 
Processing sample 25200 to 25300 
Processing sample 25300 to 25400 
Processing sample 25400 to 25500 
Processing sample 25500 to 25600 
Processing sample 25600 to 25700 
Processing sample 25700 to 25800 
Processing sample 25800 to 25900 
Processing sample 25900 to 26000 
Processing sample 26000 to 26100 
Processing sample 26100 to 26200 
Processing sample 26200 to 26300 
Processing sample 26300 to 26400 
Processing sample 26400 to 26500 
Processing sample 26500 to 26600 
Processing sample 26600 to 26700 
Processing sample 26700 to 26800 
Processing sample 26800 to 26900 
Processing sample 26900 to 27000 
Processing sample 27000 to 27100 
Processing sample 27100 to 27200 
Processing sample 27200 to 27300 
Processing sample 27300 to 27400 
Processing sample 27400 to 27500 
Processing sample 27500 to 27600 
Processing sample 27600 to 27700 
Processing sam

Processing sample 9600 to 9700 
Processing sample 9700 to 9800 
Processing sample 9800 to 9900 
Processing sample 9900 to 10000 


In [11]:
print(X_train_pt.shape)
print(y_train.shape)


(40000, 768)
(40000,)


In [22]:
inputs = tf.keras.layers.Input(shape=(n_dim), dtype=tf.float32)
outputs = tf.keras.layers.Dropout(0.1)(inputs)
outputs = tf.keras.layers.Dense(128, activation='relu')(inputs)
outputs = tf.keras.layers.Dropout(0.1)(outputs)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(outputs)
model =  tf.keras.Model(inputs, outputs)
print(model.summary())



Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 768)]             0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               98432     
_________________________________________________________________
dropout_64 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 129       
Total params: 98,561
Trainable params: 98,561
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history =  model.fit(X_train_pt, y_train, batch_size=32, epochs=8, validation_data=(X_test_pt, y_test))

Epoch 1/8
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1233/1250 [============================>.] - ETA: 0s - loss: 0.4899 - accuracy: 0.7615WARNING: AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f62643171f0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1250/1250 [==============================] - 5s 4ms/step - loss: 0.4894 - accur